In [1]:
import pandas as pd

In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from joblib import parallel_backend
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

In [3]:
train_trans=pd.read_csv("Train_trans.csv")

In [4]:
from sklearn.ensemble import IsolationForest

In [5]:
iso=IsolationForest(random_state=0)
iso.fit(train_trans)

IsolationForest(behaviour='deprecated', bootstrap=False, contamination='auto',
                max_features=1.0, max_samples='auto', n_estimators=100,
                n_jobs=None, random_state=0, verbose=0, warm_start=False)

In [6]:
pred=iso.predict(train_trans)

In [7]:
df_outlier=train_trans
df_outlier['outliers']=pred

In [8]:
df_nonOuts=df_outlier[df_outlier['outliers']!=-1]
df_nonOuts.head(5)

,avg_time_charging_lag1,avg_time_charging_lag2,avg_time_charging_lag3,avg_time_charging_lag7,charging_rate_lag3,charging_rate_lag7,avg_time_discharging_lag1,avg_time_discharging_lag2,avg_time_discharging_lag3,avg_time_discharging_lag7,...,avg_volt_change_charging,avg_volt_change_discharging,avg_time_charging,avg_time_discharging,max_voltage_day,piececount,cycle_time,use_time,fail_7,outliers
0,5.12,41.11,6.56,25.39,0.086667,-0.006667,4.37,91.70,15.70,39.12,...,377.080000,367.428750,21.696250,39.005000,4174.875000,14.200000,60.701250,543,0,1
1,36.60,5.16,6.23,6.96,0.136667,-1.296667,62.67,6.53,6.16,8.25,...,350.517857,350.309286,19.883571,32.213571,4161.000000,19.777778,52.097143,522,0,1
2,5.51,5.04,4.52,5.96,-0.460000,-0.083333,5.13,5.65,4.14,14.02,...,346.113333,343.409524,13.675238,20.471429,4152.238095,19.200000,34.146667,518,0,1
3,4.66,39.85,35.76,40.69,0.076667,-0.006667,3.93,76.37,60.86,73.53,...,356.184211,349.844737,16.306842,26.189474,4164.631579,23.625000,42.496316,548,0,1
4,5.10,43.24,4.63,5.26,0.040000,-0.153333,4.69,71.44,3.80,6.29,...,370.904000,367.730000,18.797333,30.812000,4169.800000,12.666667,49.609333,566,0,1


In [9]:
df_nonOuts_final = df_nonOuts.drop('outliers',axis=1)

In [10]:
df_nonOuts_final.head(5)

,avg_time_charging_lag1,avg_time_charging_lag2,avg_time_charging_lag3,avg_time_charging_lag7,charging_rate_lag3,charging_rate_lag7,avg_time_discharging_lag1,avg_time_discharging_lag2,avg_time_discharging_lag3,avg_time_discharging_lag7,...,number_times_restart,avg_volt_change_charging,avg_volt_change_discharging,avg_time_charging,avg_time_discharging,max_voltage_day,piececount,cycle_time,use_time,fail_7
0,5.12,41.11,6.56,25.39,0.086667,-0.006667,4.37,91.70,15.70,39.12,...,2.375000,377.080000,367.428750,21.696250,39.005000,4174.875000,14.200000,60.701250,543,0
1,36.60,5.16,6.23,6.96,0.136667,-1.296667,62.67,6.53,6.16,8.25,...,1.000000,350.517857,350.309286,19.883571,32.213571,4161.000000,19.777778,52.097143,522,0
2,5.51,5.04,4.52,5.96,-0.460000,-0.083333,5.13,5.65,4.14,14.02,...,4.095238,346.113333,343.409524,13.675238,20.471429,4152.238095,19.200000,34.146667,518,0
3,4.66,39.85,35.76,40.69,0.076667,-0.006667,3.93,76.37,60.86,73.53,...,0.736842,356.184211,349.844737,16.306842,26.189474,4164.631579,23.625000,42.496316,548,0
4,5.10,43.24,4.63,5.26,0.040000,-0.153333,4.69,71.44,3.80,6.29,...,1.133333,370.904000,367.730000,18.797333,30.812000,4169.800000,12.666667,49.609333,566,0


In [11]:
X_train_nonOuts=df_nonOuts_final.drop('fail_7',axis=1)
Y_train_nonOuts=df_nonOuts_final['fail_7']



gbdt=GradientBoostingClassifier(random_state=0)


with parallel_backend('threading', n_jobs=-1):
    score_record=cross_val_score(gbdt,X_train_nonOuts,Y_train_nonOuts,cv=StratifiedKFold(n_splits=5, random_state=0,shuffle=True),scoring='accuracy').mean()

In [12]:
score_record

0.8154441902032993

In [13]:
df_eng=df_nonOuts_final

In [14]:
import re
pattern1 = ("^avg_time_charging_lag")
match1 = []
names1 = []
for item in df_eng.columns:
    result1 = re.match(pattern1, item)
    match1.append(bool(result1))
    if bool(result1):
        names1.append(item)
names1

['avg_time_charging_lag1',
 'avg_time_charging_lag2',
 'avg_time_charging_lag3',
 'avg_time_charging_lag7',
 'avg_time_charging_lag4',
 'avg_time_charging_lag5',
 'avg_time_charging_lag6',
 'avg_time_charging_lag8',
 'avg_time_charging_lag9',
 'avg_time_charging_lag10',
 'avg_time_charging_lag11',
 'avg_time_charging_lag12',
 'avg_time_charging_lag13',
 'avg_time_charging_lag14']

In [15]:
pattern2 = ("^avg_time_discharging_lag")
match2 = []
names2 = []
for item in df_eng.columns:
    result2 = re.match(pattern2, item)
    match2.append(bool(result2))
    if bool(result2):
        names2.append(item)
names2

['avg_time_discharging_lag1',
 'avg_time_discharging_lag2',
 'avg_time_discharging_lag3',
 'avg_time_discharging_lag7',
 'avg_time_discharging_lag4',
 'avg_time_discharging_lag5',
 'avg_time_discharging_lag6',
 'avg_time_discharging_lag8',
 'avg_time_discharging_lag9',
 'avg_time_discharging_lag10',
 'avg_time_discharging_lag11',
 'avg_time_discharging_lag12',
 'avg_time_discharging_lag13',
 'avg_time_discharging_lag14']

In [16]:
pattern3 = ("^charging_rate")
match3 = []
names3 = []
for item in df_eng.columns:
    result3 = re.match(pattern3, item)
    match3.append(bool(result3))
    if bool(result3):
        names3.append(item)
names3

['charging_rate_lag3',
 'charging_rate_lag7',
 'charging_rate_lag4',
 'charging_rate_lag5',
 'charging_rate_lag6']

In [17]:
pattern4 = ("^discharging_rate")
match4 = []
names4 = []
for item in df_eng.columns:
    result4 = re.match(pattern4, item)
    match4.append(bool(result4))
    if bool(result4):
        names4.append(item)
names4

['discharging_rate_lag3',
 'discharging_rate_lag7',
 'discharging_rate_lag4',
 'discharging_rate_lag5',
 'discharging_rate_lag6',
 'discharging_rate_lag8']

In [18]:
df_eng["avg_time_charging_14"] = 0
for item in names1:
    df_eng["avg_time_charging_14"] = df_eng["avg_time_charging_14"] + df_eng[item]
df_eng["avg_time_charging_14"] = df_eng["avg_time_charging_14"] / len(names1)
df_eng["avg_time_charging_14"]

0        20.321286
1        18.247571
2        14.462143
3        15.245714
4        19.771429
           ...    
40495    13.792857
40496    18.573571
40497    14.123571
40498    14.321429
40499    18.780714
Name: avg_time_charging_14, Length: 37891, dtype: float64

In [19]:
df_eng["avg_time_discharging_14"] = 0
for item in names2:
    df_eng["avg_time_discharging_14"] = df_eng["avg_time_discharging_14"] + df_eng[item]
df_eng["avg_time_discharging_14"] = df_eng["avg_time_discharging_14"] / len(names2)
df_eng["avg_time_discharging_14"]

0        34.634286
1        28.638714
2        21.704286
3        24.710714
4        32.609286
           ...    
40495    22.824286
40496    30.192857
40497    22.622857
40498    23.245000
40499    28.634286
Name: avg_time_discharging_14, Length: 37891, dtype: float64

In [20]:
df_eng["avg_charging_rate"] = 0
for item in names3:
    df_eng["avg_charging_rate"] = df_eng["avg_charging_rate"] + df_eng[item]
df_eng["avg_charging_rate"] = df_eng["avg_charging_rate"] / len(names3)
df_eng["avg_charging_rate"]

0       -0.352000
1       -0.390667
2       -0.116667
3        0.368000
4        0.304667
           ...   
40495   -0.037333
40496   -0.202000
40497    0.347333
40498    0.115333
40499    0.300000
Name: avg_charging_rate, Length: 37891, dtype: float64

In [21]:
df_eng["avg_discharging_rate"] = 0
for item in names4:
    df_eng["avg_discharging_rate"] = df_eng["avg_discharging_rate"] + df_eng[item]
df_eng["avg_discharging_rate"] = df_eng["avg_discharging_rate"] / len(names4)
df_eng["avg_discharging_rate"]

0        0.121111
1       -1.671667
2       -0.258333
3       -0.008889
4       -0.355000
           ...   
40495    0.215556
40496    0.063333
40497   -0.678889
40498    0.011111
40499    0.017222
Name: avg_discharging_rate, Length: 37891, dtype: float64

In [22]:
df_eng.head(5)

,avg_time_charging_lag1,avg_time_charging_lag2,avg_time_charging_lag3,avg_time_charging_lag7,charging_rate_lag3,charging_rate_lag7,avg_time_discharging_lag1,avg_time_discharging_lag2,avg_time_discharging_lag3,avg_time_discharging_lag7,...,avg_time_discharging,max_voltage_day,piececount,cycle_time,use_time,fail_7,avg_time_charging_14,avg_time_discharging_14,avg_charging_rate,avg_discharging_rate
0,5.12,41.11,6.56,25.39,0.086667,-0.006667,4.37,91.70,15.70,39.12,...,39.005000,4174.875000,14.200000,60.701250,543,0,20.321286,34.634286,-0.352000,0.121111
1,36.60,5.16,6.23,6.96,0.136667,-1.296667,62.67,6.53,6.16,8.25,...,32.213571,4161.000000,19.777778,52.097143,522,0,18.247571,28.638714,-0.390667,-1.671667
2,5.51,5.04,4.52,5.96,-0.460000,-0.083333,5.13,5.65,4.14,14.02,...,20.471429,4152.238095,19.200000,34.146667,518,0,14.462143,21.704286,-0.116667,-0.258333
3,4.66,39.85,35.76,40.69,0.076667,-0.006667,3.93,76.37,60.86,73.53,...,26.189474,4164.631579,23.625000,42.496316,548,0,15.245714,24.710714,0.368000,-0.008889
4,5.10,43.24,4.63,5.26,0.040000,-0.153333,4.69,71.44,3.80,6.29,...,30.812000,4169.800000,12.666667,49.609333,566,0,19.771429,32.609286,0.304667,-0.355000


In [23]:
X_train_eng=df_eng.drop(['fail_7'],axis=1)
Y_train=df_eng["fail_7"]


gbdt=GradientBoostingClassifier(random_state=0)


with parallel_backend('threading', n_jobs=-1):
    score_record=cross_val_score(gbdt,X_train_eng,Y_train,cv=StratifiedKFold(n_splits=5, random_state=0,shuffle=True),scoring='accuracy').mean()

In [24]:
# 53 + 4 avg
score_record

0.815655369493939

In [25]:
core_features=['discharging_rate_lag3','charge_cycle_time_below_12','charging_rate_lag3','avg_volt_change_charging']

X_train_nonOuts_core=X_train_nonOuts[core_features]

gbdt=GradientBoostingClassifier(random_state=0)


with parallel_backend('threading', n_jobs=-1):
    score_record=cross_val_score(gbdt,X_train_nonOuts_core,Y_train_nonOuts,cv=StratifiedKFold(n_splits=5, random_state=0,shuffle=True),scoring='accuracy').mean()

In [26]:
X_train_nonOuts_core.head()

,discharging_rate_lag3,charge_cycle_time_below_12,charging_rate_lag3,avg_volt_change_charging
0,-1.163333,0.0,0.086667,377.080000
1,-0.066667,0.0,0.136667,350.517857
2,-2.643333,0.0,-0.460000,346.113333
3,-0.166667,0.0,0.076667,356.184211
4,0.066667,1.0,0.040000,370.904000


In [27]:
# 4 core
score_record

0.8163943507554855

In [28]:
df_eng['dummy_12_disratelag3'] = df_eng['charge_cycle_time_below_12']*df_eng['discharging_rate_lag3']

In [29]:
df_eng['dummy_12_chargeratelag3'] = df_eng['charge_cycle_time_below_12']*df_eng['charging_rate_lag3']

In [30]:
df_eng['dummy_12_avgvoltchangecharging'] = df_eng['charge_cycle_time_below_12']*df_eng['avg_volt_change_charging']

In [31]:
X_train_eng=df_eng.drop(['fail_7'],axis=1)
Y_train=df_eng["fail_7"]


gbdt=GradientBoostingClassifier(random_state=0)


with parallel_backend('threading', n_jobs=-1):
    score_record=cross_val_score(gbdt,X_train_eng,Y_train,cv=StratifiedKFold(n_splits=5, random_state=0,shuffle=True),scoring='accuracy').mean()

In [32]:
# 57 + 3 dummy
score_record

0.816552696918403

In [33]:
core_features_dummy=['discharging_rate_lag3','charge_cycle_time_below_12','charging_rate_lag3','avg_volt_change_charging',
                     'dummy_12_disratelag3', 'dummy_12_chargeratelag3', 'dummy_12_avgvoltchangecharging']

X_train_nonOuts_core_dummy=df_eng[core_features_dummy]

gbdt=GradientBoostingClassifier(random_state=0)


with parallel_backend('threading', n_jobs=-1):
    score_record=cross_val_score(gbdt,X_train_nonOuts_core_dummy,Y_train_nonOuts,cv=StratifiedKFold(n_splits=5, random_state=0,shuffle=True),scoring='accuracy').mean()

In [34]:
# 4 + 3 dummy
score_record

0.8163679516030165

In [35]:
df_eng['charging_capacity'] = df_eng['avg_charging_rate']*df_eng['avg_time_charging']

In [36]:
df_eng['discharging_capacity'] = df_eng['avg_discharging_rate']*df_eng['avg_time_discharging']

In [37]:
X_train_eng=df_eng.drop(['fail_7'],axis=1)
Y_train=df_eng["fail_7"]


gbdt=GradientBoostingClassifier(random_state=0)


with parallel_backend('threading', n_jobs=-1):
    score_record=cross_val_score(gbdt,X_train_eng,Y_train,cv=StratifiedKFold(n_splits=5, random_state=0,shuffle=True),scoring='accuracy').mean()

In [38]:
# 60 + 2 capacity
score_record

0.8166054812942278

In [39]:
core_features_capacity=['discharging_rate_lag3','charge_cycle_time_below_12','charging_rate_lag3','avg_volt_change_charging',
                     'charging_capacity', 'discharging_capacity']

X_train_nonOuts_core_capacity=df_eng[core_features_capacity]

gbdt=GradientBoostingClassifier(random_state=0)


with parallel_backend('threading', n_jobs=-1):
    score_record=cross_val_score(gbdt,X_train_nonOuts_core_capacity,Y_train_nonOuts,cv=StratifiedKFold(n_splits=5, random_state=0,shuffle=True),scoring='accuracy').mean()

In [40]:
# 4 + 2 capacity
score_record

0.8166318734821403

In [41]:
core_features_capacity_dummy=['discharging_rate_lag3','charge_cycle_time_below_12','charging_rate_lag3','avg_volt_change_charging',
                     'charging_capacity', 'discharging_capacity',
                             'dummy_12_disratelag3', 'dummy_12_chargeratelag3', 'dummy_12_avgvoltchangecharging']

X_train_nonOuts_core_capacity_dummy=df_eng[core_features_capacity_dummy]

gbdt=GradientBoostingClassifier(random_state=0)


with parallel_backend('threading', n_jobs=-1):
    score_record=cross_val_score(gbdt,X_train_nonOuts_core_capacity_dummy,Y_train_nonOuts,cv=StratifiedKFold(n_splits=5, random_state=0,shuffle=True),scoring='accuracy').mean()

In [42]:
# 4 + 2 capacity + 3 dummy
score_record

0.8161568106174389

In [43]:
df_eng['charging_capacity14'] = df_eng['avg_charging_rate']*df_eng['avg_time_charging_14']
df_eng['discharging_capacity14'] = df_eng['avg_discharging_rate']*df_eng['avg_time_discharging_14']

In [44]:
X_train_eng=df_eng.drop(['fail_7'],axis=1)
Y_train=df_eng["fail_7"]


gbdt=GradientBoostingClassifier(random_state=0)


with parallel_backend('threading', n_jobs=-1):
    score_record=cross_val_score(gbdt,X_train_eng,Y_train,cv=StratifiedKFold(n_splits=5, random_state=0,shuffle=True),scoring='accuracy').mean()

In [45]:
# 62 + 2 capacity 14
score_record

0.816869389244238

In [46]:
core_features_capacity14=['discharging_rate_lag3','charge_cycle_time_below_12','charging_rate_lag3','avg_volt_change_charging',
                          'charging_capacity14', 'discharging_capacity14']

X_train_nonOuts_core_capacity14=df_eng[core_features_capacity14]

gbdt=GradientBoostingClassifier(random_state=0)


with parallel_backend('threading', n_jobs=-1):
    score_record=cross_val_score(gbdt,X_train_nonOuts_core_capacity14,Y_train_nonOuts,cv=StratifiedKFold(n_splits=5, random_state=0,shuffle=True),scoring='accuracy').mean()

In [47]:
# 4 + 2 capacity 14
score_record

0.8166846091060675

In [48]:
df_eng['max_charging_energy_day'] = df_eng['charging_capacity']*df_eng['max_voltage_day']
df_eng['max_discharging_energy_day'] = df_eng['discharging_capacity']*df_eng['max_voltage_day']

In [49]:
X_train_eng=df_eng.drop(['fail_7'],axis=1)
Y_train=df_eng["fail_7"]


gbdt=GradientBoostingClassifier(random_state=0)


with parallel_backend('threading', n_jobs=-1):
    score_record=cross_val_score(gbdt,X_train_eng,Y_train,cv=StratifiedKFold(n_splits=5, random_state=0,shuffle=True),scoring='accuracy').mean()

In [50]:
# 64 + 2 power load
score_record

0.816869389244238

In [51]:
df_eng.head(5)

,avg_time_charging_lag1,avg_time_charging_lag2,avg_time_charging_lag3,avg_time_charging_lag7,charging_rate_lag3,charging_rate_lag7,avg_time_discharging_lag1,avg_time_discharging_lag2,avg_time_discharging_lag3,avg_time_discharging_lag7,...,avg_discharging_rate,dummy_12_disratelag3,dummy_12_chargeratelag3,dummy_12_avgvoltchangecharging,charging_capacity,discharging_capacity,charging_capacity14,discharging_capacity14,max_charging_energy_day,max_discharging_energy_day
0,5.12,41.11,6.56,25.39,0.086667,-0.006667,4.37,91.70,15.70,39.12,...,0.121111,-0.000000,0.00,0.000,-7.637080,4.723939,-7.153093,4.194597,-31883.854365,19721.854378
1,36.60,5.16,6.23,6.96,0.136667,-1.296667,62.67,6.53,6.16,8.25,...,-1.671667,-0.000000,0.00,0.000,-7.767849,-53.850354,-7.128718,-47.874384,-32322.017903,-224071.321198
2,5.51,5.04,4.52,5.96,-0.460000,-0.083333,5.13,5.65,4.14,14.02,...,-0.258333,-0.000000,-0.00,0.000,-1.595444,-5.288452,-1.687250,-5.606940,-6624.665199,-21958.913424
3,4.66,39.85,35.76,40.69,0.076667,-0.006667,3.93,76.37,60.86,73.53,...,-0.008889,-0.000000,0.00,0.000,6.000918,-0.232795,5.610423,-0.219651,24991.612175,-969.506760
4,5.10,43.24,4.63,5.26,0.040000,-0.153333,4.69,71.44,3.80,6.29,...,-0.355000,0.066667,0.04,370.904,5.726921,-10.938260,6.023695,-11.576296,23880.114729,-45610.356548
